## Install (Colab Only)

In [ ]:
# install
!pip install pyepo
!pip install mpax

Cloning into 'PyEPO'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 147 (delta 30), reused 64 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 6.94 MiB | 5.08 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Processing ./PyEPO/pkg
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1

Remove the problematic hook from Google Colab.

In [ ]:
import sys
sys.meta_path = [hook for hook in sys.meta_path if not any(keyword in str(hook) for keyword in ["google.colab"])]

In this tutorial, we will use MPAX as optimization solver to run PDHG on GPU.

[MPAX](https://github.com/MIT-Lu-Lab/MPAX) (Mathematical Programming in JAX) s a highly efficient mathematical programming framework that hardware-accelerated, differentiable, batchable, and distributable, esigned to integrate with modern computational and deep learning workflows.

At its core, MPAX leverages PDHG (Primal-Dual Hybrid Gradient), a powerful first-order optimization algorithm particularly efficient for large-scale problems. PDHG is highly compatible with batch processing and GPU acceleration, making it an ideal for deep learning.

## 1 Large Knapsack on MPAX

In [ ]:
import pyepo
# generate data
m = 10000 # number of items
k = 50    # resource dimension
n = 1000  # number of data
caps = [500] * k # capacity
p = 5     # feature dimention
deg = 4   # polynomial degree
e = 0.5   # noise half-width
weights, feats, costs = pyepo.data.knapsack.genData(num_data=n+1000, num_features=p, num_items=m,
                                                    dim=k, deg=deg, noise_width=e, seed=42)

Thus, we can use ``optMpaxModel`` to for linear programming.

In [ ]:
# build optModel
from pyepo.model.mpax import knapsackModel
optmodel = knapsackModel(weights, caps)

## 2 Dataset and Data Loader

Similar to other solver, we can use `optDataset`.

In [ ]:
# split train test data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)

In [ ]:
# get optDataset
dataset_train = pyepo.data.dataset.optDataset(optmodel, x_train, c_train)
dataset_test = pyepo.data.dataset.optDataset(optmodel, x_test, c_test)

Optimizing for optDataset...


100%|██████████| 1000/1000 [02:11<00:00,  7.63it/s]


Optimizing for optDataset...


100%|██████████| 1000/1000 [02:07<00:00,  7.86it/s]


In [ ]:
# set data loader
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

# 3 Linear Regression on PyTorch

Here, we build the simplest PyTorch model, linear regression.

In [ ]:
from torch import nn
# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(p, m)

    def forward(self, x):
        out = self.linear(x)
        return out

# 3 Training

Define function to train model with different methods.

In [ ]:
import time

# train model
def trainModel(reg, loss_func, method_name, num_epochs=10, lr=1e-2):
    # set adam optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # train mode
    reg.train()
    # init log
    loss_log = []
    loss_log_regret = [pyepo.metric.regret(reg, optmodel, loader_test)]
    # init elpased time
    elapsed = 0
    for epoch in range(num_epochs):
        # start timing
        tick = time.time()
        # load data
        for i, data in enumerate(loader_train):
            x, c, w, z = data
            # cuda
            if torch.cuda.is_available():
                x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
            # forward pass
            cp = reg(x)
            if method_name == "spo+":
                loss = loss_func(cp, c, w, z)
            if method_name == "pfy":
                loss = loss_func(cp, w)
            if method_name == "ltr":
                loss = loss_func(cp, c)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # record time
            tock = time.time()
            elapsed += tock - tick
            # log
            loss_log.append(loss.item())
        regret = pyepo.metric.regret(reg, optmodel, loader_test)
        loss_log_regret.append(regret)
        print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, loss.item(), regret*100))
    print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))
    return loss_log, loss_log_regret

## 3.1 Smart Predict-then-Optimize

In [ ]:
# init SPO+ loss
spop = pyepo.func.SPOPlus(optmodel)

Num of cores: 1


In [ ]:
import torch
# init model
reg = LinearRegression()
# cuda
if torch.cuda.is_available():
    reg = reg.cuda()
# train
loss_log, loss_log_regret = trainModel(reg, loss_func=spop, method_name="spo+")

Epoch  1,  Loss: 1051.8181,  Regret: 40.3334%
Epoch  2,  Loss: 1556.9424,  Regret: 33.3379%
Epoch  3,  Loss: 1159.5161,  Regret: 29.9910%
Epoch  4,  Loss:  662.3110,  Regret: 27.9450%
Epoch  5,  Loss:  479.3947,  Regret: 26.6849%
Epoch  6,  Loss: 1020.9800,  Regret: 25.9281%
Epoch  7,  Loss:  645.3926,  Regret: 25.4048%
Epoch  8,  Loss:  375.7114,  Regret: 25.1683%
Epoch  9,  Loss:  818.3668,  Regret: 24.9954%
Epoch 10,  Loss:  800.0767,  Regret: 24.8941%
Total Elapsed Time: 1116.01 Sec.


## 3.2 Perturbed Fenchel-Young Loss

In [ ]:
# init pfyl loss
pfy = pyepo.func.perturbedFenchelYoung(optmodel, n_samples=3, sigma=1.0)

Num of cores: 1


In [ ]:
import torch
# init model
reg = LinearRegression()
# cuda
if torch.cuda.is_available():
    reg = reg.cuda()
# train
loss_log, loss_log_regret = trainModel(reg, loss_func=pfy, method_name="pfy")

Epoch  1,  Loss:  113.6920,  Regret: 43.4046%
Epoch  2,  Loss:  110.4038,  Regret: 33.5827%
Epoch  3,  Loss:  109.1026,  Regret: 29.4995%
Epoch  4,  Loss:  106.5823,  Regret: 27.7579%
Epoch  5,  Loss:  106.6420,  Regret: 26.8516%
Epoch  6,  Loss:  105.0522,  Regret: 26.3880%
Epoch  7,  Loss:  101.1143,  Regret: 26.2023%
Epoch  8,  Loss:  100.7843,  Regret: 25.9881%
Epoch  9,  Loss:  102.8199,  Regret: 25.7716%
Epoch 10,  Loss:  104.2387,  Regret: 25.7466%
Total Elapsed Time: 4311.60 Sec.


## 3.3 Pointwise Learning To Rank

In [ ]:
# init ltr loss
ptltr = pyepo.func.pointwiseLTR(optmodel, solve_ratio=0.05, dataset=dataset_train)

Num of cores: 1


In [ ]:
import torch
# init model
reg = LinearRegression()
# cuda
if torch.cuda.is_available():
    reg = reg.cuda()
# train
loss_log, loss_log_regret = trainModel(reg, loss_func=ptltr, method_name="ltr")

Epoch  1,  Loss: 146505.3438,  Regret: 50.8165%
Epoch  2,  Loss: 76844.2031,  Regret: 43.2709%
Epoch  3,  Loss: 330145.3750,  Regret: 39.5543%
Epoch  4,  Loss: 121497.7812,  Regret: 36.8107%
Epoch  5,  Loss: 66077.6719,  Regret: 35.3827%
Epoch  6,  Loss: 125274.8438,  Regret: 35.1007%
Epoch  7,  Loss: 44842.1797,  Regret: 34.6702%
Epoch  8,  Loss: 32069.6445,  Regret: 34.1945%
Epoch  9,  Loss: 43524.7305,  Regret: 33.7943%
Epoch 10,  Loss: 46142.7422,  Regret: 33.2205%
Total Elapsed Time: 104.54 Sec.
